In [4]:
# Helper funcitons like these are important because the input format varies between the provided models
# Using the below 'helper' function we can account for changes to the input JSON

import json
import boto3

def invoke_model(client, prompt, model, 
    accept = 'application/json', content_type = 'application/json',
    max_tokens  = 512, temperature = 1.0, top_p = 1.0, top_k = 250, stop_sequences = [],
    count_penalty = 0, presence_penalty = 0, frequency_penalty = 0, return_likelihoods = 'NONE'):
    # default response
    output = ''
    # identify the model provider
    provider = model.split('.')[0] 
    # InvokeModel
    if (provider == 'anthropic'): 
        user_message = {'role': 'user', 'content': prompt}
        messages = [user_message]
        input = {
            'anthropic_version': 'bedrock-2023-05-31',
            'system': '',
            'messages': messages,
            'max_tokens': max_tokens, 
            'temperature': temperature,
            'top_k': top_k,
            'top_p': top_p,
            'stop_sequences': stop_sequences
        }
        body = json.dumps(input)
        response = client.invoke_model(body=body, modelId=model, accept=accept, contentType=content_type)
        response_body = json.loads(response.get('body').read())
        completions = response_body['content']
        for part in completions:
            output = output + part['text']
    elif (provider == 'ai21'): 
        input = {
            'prompt': prompt, 
            'maxTokens': max_tokens,
            'temperature': temperature,
            'topP': top_p,
            'stopSequences': stop_sequences,
            'countPenalty': {'scale': count_penalty},
            'presencePenalty': {'scale': presence_penalty},
            'frequencyPenalty': {'scale': frequency_penalty}
        }
        body = json.dumps(input)
        response = client.invoke_model(body=body, modelId=model, accept=accept, contentType=content_type)
        response_body = json.loads(response.get('body').read())
        completions = response_body['completions']
        for part in completions:
            output = output + part['data']['text']
    elif (provider == 'amazon'): 
        input = {
            'inputText': prompt,
            'textGenerationConfig': {
                  'maxTokenCount': max_tokens,
                  'stopSequences': stop_sequences,
                  'temperature': temperature,
                  'topP': top_p
            }
        }
        body = json.dumps(input)
        response = client.invoke_model(body=body, modelId=model, accept=accept, contentType=content_type)
        response_body = json.loads(response.get('body').read())
        results = response_body['results']
        for result in results:
            output = output + result['outputText']
    elif (provider == 'cohere'): 
        input = {
            'prompt': prompt, 
            'max_tokens': max_tokens,
            'temperature': temperature,
            'k': top_k,
            'p': top_p,
            'stop_sequences': stop_sequences,
            'return_likelihoods': return_likelihoods
        }
        body = json.dumps(input)
        response = client.invoke_model(body=body, modelId=model, accept=accept, contentType=content_type)
        response_body = json.loads(response.get('body').read())
        results = response_body['generations']
        for result in results:
            output = output + result['text']
    elif (provider == 'meta'): 
        input = {
            'prompt': prompt,
            'max_gen_len': max_tokens,
            'temperature': temperature,
            'top_p': top_p
        }
        body = json.dumps(input)
        response = client.invoke_model(body=body, modelId=model, accept=accept, contentType=content_type)
        response_body = json.loads(response.get('body').read())
        output = response_body['generation']
    # return
    return output

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
model  = 'anthropic.claude-3-sonnet-20240229-v1:0'
prompt = """
Explain how chicken swim to an 8 year old using 2 paragraphs.
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)


Here's how you can explain to an 8-year-old how chickens swim, using two paragraphs:

Paragraph 1:
Chickens are birds, and they are not really built for swimming. Their bodies are covered in feathers, which are lightweight and help them fly, but they can get heavy and waterlogged when wet. Chickens also have small, scaly legs and feet, which are designed for walking and scratching the ground, not paddling through water. However, in some situations, chickens may find themselves in water and need to swim to safety.

Paragraph 2:
When a chicken ends up in water, it will use its wings to help it stay afloat and move forward. The chicken will flap its wings rapidly, creating a paddling motion that propels it through the water. Its feet will also kick and help with the swimming motion. Chickens can swim for short distances, but they quickly become tired and need to get out of the water. If they are in a pond or a shallow body of water, they may be able to walk or run along the bottom to get 

In [2]:
# Changing the model
model  = 'ai21.j2-ultra'
prompt = """
Explain how chicken swim to an 8 year old using 2 paragraphs.
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model, temperature=0.9)
print(output)


Birds are noted for their flying prowess, and for good reason. But have you noticed that some birds also are powerful swimmers? Chickens are kind of funny though; when they swim, they sort of move their chicken feet up and down quickly, so they can "swim" across the water. If they need to get to shore from a boat, they will often use their chicken feet to paddle to the riverbank. Chickens have no wings and no arms, so of course they can't fly. But their chickens feet are so strong and clever, they can swim by themselves!


In [3]:
# Adding Parameters for LLM

model  = 'ai21.j2-ultra'
prompt = """
Explain how chicken swim to an 8 year old using 2 paragraphs.
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model, temperature=0.9)
print(output)

Chickens are birds, and birds normally don't swim. However, some species of chickens are good at swimming. They may look like typical chickens, but they have waterproof feathers and webbed feet. These feathers and webbed feet help the chickens to paddle and move through the water. When these chickens need to swim, they flap their wings and create strong currents that propel them forward. They also use their webbed feet to kick and swim through the water.


In [12]:
# Parameter Tuning

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)

model  = 'ai21.j2-ultra'
prompt = """
Name a few house pets with 4 legs.
"""
output = invoke_model(
    client=bedrock, prompt=prompt, model=model, 
    temperature=1.0, top_p=1.0, max_tokens=512, stop_sequences=[])
print(output)

Some house pets with four legs are dogs, cats, and hamsters.


In [11]:
# Adding to Stop Sequences

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)

model  = 'ai21.j2-ultra'
prompt = """
Name a few house pets with 4 legs.
"""
output = invoke_model(
    client=bedrock, prompt=prompt, model=model, 
    temperature=1.0, top_p=1.0, max_tokens=512, stop_sequences=["cat"])
print(output)

Some house pets with four legs are dogs, 


In [10]:
# Adding Paramters (top_k=250) --> Top K defines the cut off where the LLM model no longer selects the words
model  = 'anthropic.claude-3-sonnet-20240229-v1:0'
prompt = """
Name a few house pets with 4 legs.
"""
output = invoke_model(
    client=bedrock, prompt=prompt, model=model, temperature=1.0, top_p=1.0, top_k=250)
print(output)


Here are a few common house pets with four legs:

- Dogs
- Cats
- Rabbits
- Guinea pigs
- Ferrets
- Hamsters
- Gerbils
- Rats
- Mice
- Chinchillas

Many small mammals that are kept as pets have four legs. Reptiles like lizards and snakes don't have legs, and birds like parrots have two legs, so they wouldn't be included in this category of four-legged house pets.


In [14]:
# 'Zero Shot' prompting. (known for giving a wide variety of outputs, due to unclear directions)
model  = 'ai21.j2-ultra'
prompt = """
A spider 
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)


I first saw the spider when I was walking through my garden
A spider is a type of invertebrate animal that is known for its eight legs, and web-spinning abilities. They can be found in a variety of environments, including gardens.


In [15]:
# 'Zero Shot' prompting. (known for giving a wide variety of outputs, due to unclear directions)
model  = 'ai21.j2-ultra'
prompt = """
A spider 
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

A spider is an arthropod that
 8


In [16]:
# 'Zero Shot' prompting. (known for giving a wide variety of outputs, due to unclear directions)
model  = 'ai21.j2-ultra'
prompt = """
A spider 
"""
output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

is an eight-legged creature
that is related to insects.
Spiders are known for their excellent
ability to spin silk webs.
Spiders are found all over the world,
and they play an important
role in the food chain.
Spiders can be beneficial
to humans, for example, by
controlling insect populations.
However, some spiders can also
be harmful to humans,
and some people may be allergic
to their venom.
As a result, it is important to be
cautious around spiders and
to seek medical attention if you
are bitten or stung by one.
.
Spiders are commonly found in homes, and they can be a nuisance. It is important to keep spiders out of your home by using screens on windows and doors, and by sealing cracks and crevices. You can also keep spiders away by using insecticides and traps.


In [17]:
# Few-Shot Prompting (Still unclear, but with a pattern. Increases consistency in response)
model  = 'ai21.j2-ultra'
prompt = """
A dog has 4 legs.
A sheep has 4 legs.
A spider
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)


Has 8 legs.


In [18]:
# Few-Shot Prompting (Still unclear, but with a pattern. Increases consistency in response)
model  = 'ai21.j2-ultra'
prompt = """
A dog has 4 legs.
A sheep has 4 legs.
A spider
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

A spider is different from sheep and dogs, as it does not have four legs. A spider has eight legs.


In [19]:
model  = 'ai21.j2-ultra'
prompt = """
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

Negative


In [20]:
model  = 'ai21.j2-ultra'
prompt = """
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)


Negative


In [21]:
model  = 'ai21.j2-ultra'
prompt = """
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

Negative


In [23]:
# Inversting the sentiments produces incorrect output

model  = 'ai21.j2-ultra'
prompt = """
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Negative
What a horrible show! //
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

Positive


In [28]:
# Let’s improve the prompt by adding the intermediate reasoning steps. 
# Now all model knows we want to first infer the sentiment from the sentence then flip it to the opposite.

model  = 'ai21.j2-ultra'
prompt = """
This is awesome! // Positive => Negative
This is bad! // Negative => Positive
Wow that movie was rad! // Positive => Negative
What a horrible show! //
"""

output = invoke_model(client=bedrock, prompt=prompt, model=model)
print(output)

This is amazing! // Positive => Negative
